# 使用朴素贝叶斯分类器从个人广告中获取区域倾向

    这个例子我们将分别从美国的两个城市中选取一些人，通过分析这些人发布的征婚广告信息，来比较这两个城市的人们在广告用词上是否不同。如果结论确实不同，那么他们各自常用词是哪些？从人们的用词当中，我们能否对不同城市的人所关心的内容有所了解。

使用朴素贝叶斯分类器来发现地域相关的用词

    （1）收集数据：从RSS源收集内容，这里需要对RSS源构建一个接口
    （2）准备数据：将文本文件解析成词条向量
    （3）分析数据：检查词条确保解析的重要性
    （4）训练算法：使用训练函数trainNB0
    （5）测试算法：观察错误率，确保分类器可用。可以修改切分程序，以降低错误率，提高分类结果
    （6）使用算法：构建一个完整的程序，封装所有内容。给定两个RSS源，该程序会显示到最常用的公共词。

使用RSS源需要先安装feedparse

In [1]:
from numpy import *
import operator
import feedparser
#构建新的词汇表，去除重复元素
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet|set(document)
    return list(vocabSet)

In [2]:
#词袋模型
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [3]:
#朴素贝叶斯分类器训练函数
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords);p1Num = ones(numWords)
    p0Denom = 2.0;p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num/p1Denom)
    p0Vect = log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

In [4]:
#朴素贝叶斯分类算法
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify*p1Vec)+log(pClass1)
    p0 = sum(vec2Classify*p0Vec)+log(1-pClass1)
    if p1>p0:
        return 1
    else:
        return 0

In [5]:
#高频词去除函数
def calcMostFreq(vocabList,fullText):
    import operator
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.count(token) #计算出现频率
    sortedFreq = sorted(freqDict.items(),key=operator.itemgetter(1),reverse = True)
    return sortedFreq[:30]   

In [6]:
#RSS源分类器
#使用两个RSS源作为参数
def localWords(feed1,feed0):
    import feedparser
    docList = [];classList = [];fullText = [] #文档矩阵，文档类别，所有单词
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordLlist)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    #去掉高频词汇
    top30Words = calcMostFreq(vocabList,fullText)
    for pairW in top30Words:
        if pairW[0] in vocabList:
            vocabList.remove(pairW[0])
    trainingSet = range(2*minLen)
    testSet = []
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(list(trainingSet)[randIndex])
    trainMat = [];trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(bagOfWords2VecMN(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList,docList[docIndex])
        if classifyNB(array(word2Vector),p0V,p1V,pSpam)!=classList[docIndex]:
            errorCount += 1
    print('the error rate is:',float(errorCount)/len(testSet))
    return vocabList,p0V,p1V

In [7]:
'''
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')
#为了测试精度，一般测试多次，取平均值
vocabList,pSF,pNY = localWords(ny,sf)
'''

"\nny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')\nsf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')\n#为了测试精度，一般测试多次，取平均值\nvocabList,pSF,pNY = localWords(ny,sf)\n"

In [8]:
#分析数据，显示地域相关的用词

#最具表征性的词汇显示函数
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))  # 元组列表
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))  # 元组列表
    sortedSF = sorted(topSF, key= lambda pair: pair[1], reverse= True)  # 元组列表排序
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])

In [9]:
'''
getTopWords(ny,sf)
'''

'\ngetTopWords(ny,sf)\n'

# 这段代码有问题，

    ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
    len(ny['entries'])
    显示为0